# Data Management and Database Basics

## Motivation

<img src="https://preview.redd.it/gph4rp6drvo41.jpg?width=640&crop=smart&auto=webp&s=a407a7be1da73ba010f0295a6351ab9d14471b2a" width=400 />

## Overview

1. Pre-SQL  (Robin)
2. SQL databases  (/Emilio)
3. Non-SQL databases  (Ali)
4. Porject (Ali)

# Pre-SQL

- You kind of have data, but not really that much.
- You want to organize it better,  but keep things lightweight to share.

## Working with CSV files

### Efficiently reading last lines

# SQL

## Introduction
 - SQL is a declarative programming language to manipulate tables
   - no functions or loops, just _declare_ what you need and the runtime will figure out how to compute it
 - SQL queries can be used to
   - Insert new rows into a table
   - Delete rows from a table
   - Ipdate one or more attributes of one or more rows in a table
   - Retrieve and possibly transform rows combing from one or more tables
 - Relational Database Management System (RDBMS)
   - Manages data in the tables
   - Executes queries, returns results
 - This section will mostly focus on reading data (last point)

## Main abstraction: Tables
 - A table is a _set_ of tuples (rows)
   - No two rows are the same
 - Rows are distinguished by _primary keys_
   - Primary key: smallest set of attributes that uniquely identifies a row
   - Cannot have two rows with the same primary key
   - Examples:
     - Student ID (one attribute)
     - First name, last name, birth date, place of birth (four attributes)
   - The primary key is a property of each table
     - All rows in a table use the same attributes as primary key
     - But different tables can have different primary keys

## Domain
 - Good database design has
   - One table for each entity in the domain
   - Relationships between two or more entities
 - _Foreign keys_ are used to refer to rows of other tables
   - e.g. a table with grades will have foreign keys that point to the student and the course

### Example: University
 - Entities
   - Students (ID, Name, Degree)
   - Courses (ID, Title, Faculty, Semester)
   - Professors (ID, Name, Chair)
 - Relationships
   - One student can *Mentor* another student
   - A student *Attends* several courses and obtains a grade for each of them
   - Professors *Teach* courses

### ER diagram
 - Graphical form to represent entities and relationships
   - Box: entity
   - Diamond: relationship
   - Circle: attribute
 
 
![](../img/sql_er_diagram.png)

### Which tables to create?
 - Until now, we separated entities from relationships
 - But in practice everything must be stored into tables
 - How to do this?
   - One table per entity (students, courses, professors)
   - What about the relationships?
     - Mentor: 1 to 1, three possibilities
       1. Have a column "mentor"
       2. Have a column "mentee"
          - Having both is not ideal: more work to ensure consistency
       3. Have a new table (mentor, mentee)
     - Attends: M to N
       - Requires a table (student, course)
     - Teaches: 1 to N
       - Store professor in course table or create separate table
   - General rule:
     - Using a separate table is always possible, or
     - 1 to 1: can store in either entity
     - 1 to N: store in entity with cardinality N
     - M to N: must use separate table

### Final list of tables
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)
 - Which attributes are primary and foreign keys?

## Purpose of SQL
 - SQL shines when "navigating" across relationships, for example:
   - For each student, find the professor that gave them the highest grade
   - For each professor, find courses taught last semester
 - Also used to modify data, tables, databases, etc.
   - Not discussed in this course

## Anatomy of a SELECT query
 - SELECT queries are used to retrieve data from the database
 - The result is itself a table (not saved unless specified)

```
SELECT <columns and transformation>
FROM <source table(s)>
[WHERE <filter rows coming from the source table(s)>]
[GROUP BY <create groups of rows>
[HAVING <filter groups>]]
[ORDER BY <columns> [ASC|DESC]];
```

 - Must have SELECT and FROM
 - WHERE and GROUPBY are optional
 - HAVING is optional, and must be used with GROUP BY
 - GROUP BY: eventually you must have only one row per group

## Example

Find all courses held in the Winter semester 2019/2020:

```sql
SELECT *
FROM Courses
WHERE Semester = 'WiSE 19/20';
```

## Select query untangled
 - Confusingly, the execution order is different than the writing order:
   1. FROM: first, gather all input rows from all tables
   2. WHERE: next, remove all rows not matching the predicate
   3. GROUP BY: now, if needed, create groups of rows
   4. HAVING: then, remove all groups that do not match the predicate
   5. ORDER BY: sort the tuples by a the value of a certain column
   6. SELECT: finally, produce output columns

## Interactive SQL console

An interactive SQL console with a few tables can be accessed at [w3schools.com](https://www.w3schools.com/sql/trysql.asp?filename=trysql_select_all)

 - Go to w3schools.com
 - Scroll until SQL, on the left side there will be a query and a button "Try it Yourself"
 - I encourage you to fiddle around while I am explaining
 - They also have a (superficial) command reference
 
![](../img/w3trysql.png)

## Interactive SQL console

![](../img/w3sqled.png)

## FROM: source tables
 - You can specify one or more tables in the from clause
 - FROM will do a cross-product of all tuples of all tables
 - In almost all cases, you only want a small subset of the cross-product
   - Use WHERE to remove tuples that do not make sense
 - Possible to give aliases to tables and use that alias in the rest of the query
   - Useful to keep query short and to disambiguate when the same table is used several times in the same query

## WHERE: tuple filter
 - Specify a boolean condition that is evaluated for each row produced by the FROM
 - All rows where this evaluates to false are discarded
 - Example: Associate to each student all its grades (one per row)

```sql
SELECT *
FROM
    Students AS s,
    Attend AS a,
    Course AS c
WHERE
    s.ID = a.Student
    AND a.Course = c.ID;
```

## WHERE: handling of NULL values 
 
 - NULL is used for "undefined" values
 - Nothing is equal to NULL (not even NULL)
   - `x = NULL` always equals NULL (i.e. false)
 - Use instead `x IS NULL` or `x IS NOT NULL`
 - Nasty example: `SELECT * FROM table WHERE x = 10 OR NOT x = 10`
   - When `x` contains NULLs this equals `WHERE x IS NOT NULL`
   - Dumb fix: `WHERE x = 10 OR NOT x = 10 OR x IS NULL`

## JOIN: a special case of FROM+WHERE
 - In most cases, we are not interested in the cross-product
 - We actually want tuples that match primary/foreign keys
 - This operation is so common that it has a special name to distinguish it from the general case
 - Other than the name, the two are completely equivalent
 - Join makes your intentions clearer
 - The previous query becomes:

```sql
SELECT *
FROM
    Students AS s
    JOIN Attend AS a
        ON s.ID = a.Student
    JOIN Course AS c
        ON c.ID = a.Course;
```

## Non-matching rows in JOINs
 - Options to handle non-matches:
   - Inner join: Only keep matches
   - Left join: keep matches and un-matched records from _left_ table
   - Right join: keep matches and un-matched records from _right_ table
   - Outer join: keep matches, cross-product between un-matched records
 - Other possibilities:
    - Natural join (`ON` is missing): match all columns with the same name
    - Self-join: A table with itself (e.g. to find a student's mentor)

### INNER JOIN

```sql
FROM Students [INNER] JOIN Attend
    ON Student.ID = Attend.Student
```

![](../img/sql_join_inner.svg)

### LEFT JOIN

```sql
FROM Students LEFT JOIN Attend
    ON Student.ID = Attend.Student
```

![](../img/sql_join_left.svg)

### RIGHT JOIN

```sql
FROM Students RIGHT JOIN Attend
    ON Student.ID = Attend.Student
```

![](../img/sql_join_right.svg)

### OUTER JOIN

```sql
FROM Students OUTER JOIN Attend
    ON Student.ID = Attend.Student
```

![](../img/sql_join_outer.svg)


Warning: cross-product between unmatched rows!

### Retrieving un-matched rows only

 - Example: find all students who have not attended any course

```sql
SELECT Students.ID
FROM Students LEFT JOIN Attend
    ON Students.ID = Attends.Student
WHERE
    Attends.Student IS NULL
```

![](../img/sql_join_unmatched_only.svg)

## GROUP BY: create groups of rows
 - must specify one or more columns, possibly with transformation
 - all rows that have the same values for all (transformed) column(s) end up in the same group

## HAVING: filter groups
 - A boolean condition applied to each group
 - Example: filter by group size, min/max/average of something..
 - Common case: counting
   - `COUNT(*)`: number of rows in the group
   - `COUNT(expr)`: number of rows where `expr` is not NULL
   - `COUNT(DISTINCT expr)`: number of unique values of `expr` (excluding NULLs)

## ORDER BY: order tuples

 - Sort the tuples produced by the query
 - Sort by the value of one or more columns, possibly transformed
 - Possible to order by aggregations (count/min/max/sum/avg)

## SELECT: produce output columns
 - All the surviving groups/rows are transformed
 - Select only a subset of attributes, or transform values
 - Careful: each group must be collapsed into a row

# Examples

## Example 1

Find the ID of all students who failed at least one exam.

```sql
SELECT ...
```

Tables:
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)

## Example 1

Find the ID of all students who failed at least one exam.

```sql
SELECT Student
FROM Attends
WHERE Grade > 5
```


Tables:
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)

## Example 2

Find how many exams each student failed.

```sql
SELECT ...
```

Tables:
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)

## Example 2

Find how many exams each student failed.

```sql
SELECT Student, COUNT(*)
FROM Attends
WHERE Grade > 5
GROUP BY Student
```


Tables:
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)

## Example 3

Find how many exams each student failed, only for the students who failed at least 2.

```sql
SELECT ...
```


Tables:
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)

## Example 3

Find how many exams each student failed, only for the students who failed at least 2.


```sql
SELECT Student, COUNT(*)
FROM Attends
WHERE Grade > 5
GROUP BY Student
HAVING COUNT(*) > 1
```


Tables:
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)

## Example 4

Find how many courses each student failed, only for the students who failed at least 2 exams.

```sql
SELECT ...
```


Tables:
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)

## Example 4

Find how many courses each student failed, only for the students who failed at least 2 exams.

```sql
SELECT Student, COUNT(DISTINCT Course)
FROM Attends
WHERE Grade > 5
GROUP BY Student
HAVING COUNT(*) > 1
```


Tables:
 - Students(ID, Name, Degree, Mentor)
 - Professors(ID, Name, Chair)
 - Courses(ID, Title, Faculty, Semester, Professor)
 - Attends(Student, Course, Grade)

# Transactions and ACID properties

 - When the data is read and modified by several clients at the same time, care must be taken
 - Read/modify/write workflows especially vulnerable
 - Transaction: a set of queries (reads and/or writes)
   - Atomicity: sequence of operations appears as as a single operation on the data
     - Either all operations succeed, or the all modifications are undone
   - Consistency: database invariants are always satisfied regardless of the outcome
      - Invariants: uniqueness, non-empty values, primary/foreign keys, etc.
   - Isolation: different transactions cannot "see" each other
      - Order of transactions does not matter
   - Durability: once completed, the modifications are permanent
      - Useful in case of crashes
  - All of this is handled automatically by the DBMS
    - Users only need to declare start/end and outcome of the transaction

# Interfacing to a RDBMS

Three types of clients

 1. Command line clients
 2. Graphical clients
 3. Programmatic access

## Command line clients

Enter SQL queries and administrative commands directly from the command line:

```
$ sqlite3
SQLite version 3.32.3 2020-06-18 14:00:33
Enter ".help" for usage hints.
Connected to a transient in-memory database.
Use ".open FILENAME" to reopen on a persistent database.
sqlite> 
```

```
$ psql -U user -h 10.0.6.12 -p 21334 -d database
psql (11.1, server 11.0)
Type "help" for help.

postgres=# 
```

## Graphical clients

Database-specific:
 - pgAdmin (PostgreSQL)
 - SQLite Browser (SQLite)
 - MySQL Workbench (MySQL)

General purpose:
 - [SQuirreL](http://squirrel-sql.sourceforge.net)
 - [SQLAdmin](http://sqladmin.sourceforge.net/)

### SQuirreL example: querying
![](http://squirrel-sql.sourceforge.net/screenshots/15_edit_result.png)

### SQuirreL example: visualizing tables

![](http://squirrel-sql.sourceforge.net/screenshots/7_graph.png)

## Programmatic Access

Two types of APIs:

 1. High-level: Object-relational mapping (ORM)
    - Each table has a corresponding class in the code
    - Operations on objects are automatically translated on queries
    - These libraries can work with many SQL databases
 2. Low-level: Directly write SQL queries as strings
    - Usually tied to a specific type of SQL database

### SQLAlchemy: ORM in Python

Example from [pythoncentral.io](https://www.pythoncentral.io/overview-sqlalchemys-expression-language-orm-queries/).

Tables:

```python
class Department(Base):
    __tablename__ = 'department'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    employees = relationship('Employee', secondary='department_employee')

class Employee(Base):
    __tablename__ = 'employee'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    departments = relationship('Department', secondary='department_employee')

class DepartmentEmployee(Base):
    __tablename__ = 'department_employee'
    department_id = Column(Integer, ForeignKey('department.id'), primary_key=True)
    employee_id = Column(Integer, ForeignKey('employee.id'), primary_key=True)
```

### Inserting data in SQLAlchemy

```python
from sqlalchemy import create_engine
engine = create_engine('sqlite:///')

from sqlalchemy.orm import sessionmaker
session = sessionmaker()
session.configure(bind=engine)
Base.metadata.create_all(engine)

s = session()
john = Employee(name='john')
s.add(john)
it_department = Department(name='IT')
it_department.employees.append(john)
s.add(it_department)
s.commit()
```

### Querying in SQLAlchemy

asd

```python
find_marry = select([
    Employee.id
]).select_from(
    Employee.__table__.join(DepartmentEmployee)
).group_by(
    Employee.id
).having(func.count(
    DepartmentEmployee.department_id
) > 1)

rs = s.execute(find_marry) 
rs.fetchall()  # result: [(2,)]
```

### Accessing SQLite in Python

Example from [python3.org](https://docs.python.org/3/library/sqlite3.html)

```python
import sqlite3
conn = sqlite3.connect('example.db')

c = conn.cursor()

c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

conn.commit()

conn.close()
```

### Querying SQLite in Python

```python
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00)]

c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

for row in c.execute('SELECT * FROM stocks WHERE price<? ORDER BY price', [2000,]):
    print(row)

# ('2006-03-28', 'BUY', 'IBM', 1000, 45.0)
# ('2006-04-06', 'SELL', 'IBM', 500, 53.0)
# ('2006-04-05', 'BUY', 'MSFT', 1000, 72.0)
```

## Programmatic access to SQL databases

Common concepts:

| SQLite | SQLAlchemy | Purpose |
|-|-|-|
| Connection | Engine | The database object |
| Cursor | Session | A transaction |

General workflow:

 1. Obtain a connection
    - Use a connection pool if performance is a concern
 2. Obtain a cursor
 3. Execute queries
 4. Close cursor
 5. Possibly repeat...
 6. Close connection

### SQL injection

 - An once-popular cyber-attack on SQL databases
 - Caused by improper escaping of arguments coming from external users (e.g. in a web form)
 - Never trust user input!

Example:

```python
def find_courses(conn, semester):
    c = conn.cursor()
    return c.execute(
        "SELECT * FROM Courses WHERE Semester={}".format(semester)
    ).fetchall()

# later on...
find_courses(conn, "''; DROP TABLE Courses; --")
```

This will execute **two** queries:

```sql
SELECT * FROM Courses WHERE Semester='';
DROP TABLE Courses; --
```


### Avoiding SQL injection

Let the API handle escaping for you:

```python
def find_courses(conn, semester):
    c = conn.cursor()
    return c.execute(
        "SELECT * FROM Courses WHERE Semester=?",
        [semester]
    ).fetchall()

# later on...
find_courses(conn, "''; DROP TABLE Courses; --")
```

This will execute **one** query:

```sql
SELECT * FROM Courses WHERE Semester='''''; DROP TABLE Courses; --'
```

# Where should I start from?

[SQLite](https://sqlite.org/index.html) is very simple and scales well.

[PostgreSQL](https://www.postgresql.org/) for more complicated requirements / large scale data processing.

Also frequently used: [MySQL](https://www.mysql.com/)

# Advanced topics

## Indexing
 - depending on your query and how you express it, it may be quite slow
 - the DBMS tries to optimize every query, but sometimes it fails
 - when most of the time is spent on joins and lookups, creating _indices_ can greatly speed up the query
 - an index is just a mapping from values to rows that contain that value in one or more columns
 - this makes it much faster to find rows that contain a given value
   - instead of checking row by row, simply look in the index
   - think about books!
 - an index is always relative to a table and one or more columns
   - `CREATE INDEX <index name> ON <table name>(<list of columns>)`
 - a table can have many indices
   - an index is always created automatically for primary keys
   - all other unique keys must also have an index
   - indices on foreign keys _might_ be useful
   - WHERE/JOIN are much faster when there is an index on one of the columns
 - if a query is slow and/or executed very frequently, consider adding an index on columns used in the WHERE/JOIN

## Main types of index
 - Tree-based: O(log N) access, can be used to quickly answer queries like `WHERE L < column < U`
   - Branching factor in the order of 1000s
 - Hash-based: O(1) access, cannot answer range queries
 - Clustered index: table is physically sorted by the columns

## Query plans
 - understanding why a query is slow is not trivial
 - the query plan is produced by the optimizer and shows exactly what and how is done to execute the query
 - it contains an estimated cost and can be augmented with the actual cost measured when executing the query
 - estimated cost:
   - computed from statistics about rows/values that the DBMS maintains internally
   - these statistics can become inaccurate after lots of operations
   - useful to periodically recompute these statistics
   - also useful to periodically clear the space allocated to deleted rows and defragment table data
 - (show example of plans before/after adding an index)

### Example

![](../img/qplan.png)

Image from [dba.stackexchange.com](https://dba.stackexchange.com/q/9234)

# Non-SQL

What does NoSQL actually mean?

A bit of history …
- 1998: First used for a relational database that omitted usage of SQL
- 2009: First used during a conference to advocate non-relational databases

NoSQL is an accidental term with no precise definition

## NoSQL: Overview

Main objective: implement distributed state

- Different objects stored on different servers
- Same object replicated on different servers

Main idea: give up some of the ACID constraints to improve performance

Simple interface:

- Write (=Put): needs to write all replicas
- Read (=Get): may get only one

Eventual consistency <- Strong consistency

## NoSQL Six key features

1. Scale horizontally “simple operations”
2. Replicate/distribute data over many servers
3. Simple call level interface (contrast w/ SQL)
4. Weaker concurrency model than ACID
5. Efficient use of distributed indexes and RAM
6. Flexible schema

## Types of NoSQL Databases

Core types
- Key-value stores
- Document stores
- Wide column (column family, column oriented, …) stores
- Graph databases

Non-core types
- Object databases
- Native XML databases
- RDF stores
- ...

## Key-Value Stores

Data model
- he most simple NoSQL database type
    - Works as a simple hash table (mapping)
- Key-value pairs
    - Key (id, identifier, primary key)
    - Value: binary object, black box for the database system

Query patterns
- Create, update or remove value for a given key
- Get value for a given key

Characteristics
- Simple model ⇒ great performance, easily scaled, …
- Simple model ⇒ not for complex queries nor complex data


### Key-Value Stores

Suitable use cases
- Session data, user profiles, user preferences, shopping carts, …
I.e. when values are only accessed via keys

When not to use
- Relationships among entities
- Queries requiring access to the content of the value part
- Set operations involving multiple key-value pairs

Representatives
- Redis, MemcachedDB, Riak KV, Hazelcast, Ehcache, Amazon, SimpleDB, Berkeley DB, Oracle NoSQL, Infinispan, LevelDB, Ignite, Project Voldemort
- Multi-model: OrientDB, ArangoDB

### Key-Value Stores

<img src="photos/keyvalue_example.PNG">

### Key-Value Stores Use cases

Key-value data stores are ideal for storing user profiles, blog comments, product recommendations, and session information.

- Twitter uses Redis to deliver your Twitter timeline
- Pinterest uses Redis to store lists of users, followers, unfollowers, boards, and more
- Coinbase uses Redis to enforce rate limits and guarantee correctness of Bitcoin transactions
- Quora uses Memcached to cache results from slower, persistent databases

## Document Stores

Data model
- Documents
    - Self-describing
    - Hierarchical tree structures (JSON, XML, …)
        – Scalar values, maps, lists, sets, nested documents, …
    - Identified by a unique identifier (key, …)
- Documents are organized into collections

Query patterns
- Create, update or remove a document
- Retrieve documents according to complex query conditions

Observation
- Extended key-value stores where the value part is examinable!


## Document Stores

Suitable use cases
- Event logging, content management systems, blogs, web, analytics, e-commerce applications, …
    - I.e. *for structured documents with similar schema*

When not to use
- *Set operations* involving multiple documents
- Design of document structure is constantly changing
    - I.e. when the required level of granularity would outbalance the advantages of aggregates

Representatives
- MongoDB, Couchbase, Amazon DynamoDB, CouchDB, RethinkDB, RavenDB, Terrastore
- Multi-model: MarkLogic, OrientDB, OpenLink Virtuoso, ArangoDB


## Document Stores

```json
[
    {
        "year" : 2013,
        "title" : "Turn It Down, Or Else!",
        "info" : {
            "directors" : [ "Alice Smith", "Bob Jones"],
            "release_date" : "2013-01-18T00:00:00Z",
            "rating" : 6.2,
            "genres" : ["Comedy", "Drama"],
            "image_url" : "http://ia.media-imdb.com/images/N/O9ERWAU7FS797AJ7LU8HN09AMUP908RLlo5JF90EWR7LJKQ7@@._V1_SX400_.jpg",
            "plot" : "A rock band plays their music at high volumes, annoying the neighbors.",
            "actors" : ["David Matthewman", "Jonathan G. Neff"]
        }
    },
    {
        "year": 2015,
        "title": "The Big New Movie",
        "info": {
            "plot": "Nothing happens at all.",
            "rating": 0
        }
    }
]
```

## Document Stores Use Cases

- SEGA uses MongoDB for handling 11 million in-game accounts
- Cisco moved its VSRM (video session and research manager) platform to Couchbase to achieve greater scalability
- Aer Lingus uses MongoDB with Studio 3T to handle ticketing and internal apps
- Built on MongoDB, The Weather Channel’s iOS and Android apps deliver weather alerts to 40 million users in real-time

## Wide Column Stores

Data model
- Column family (table)
    - Table is a collectioon of similar rows (not necessarily identical)

- Row
    - Row is a collectioon of columns
        - Should encompass a group of data that is accessed together
    - Associated with a unique row key

- Column
    - Column consists of a column name and column value (and possibly other metadata records)
    - Scalar values, but also flat sets, lists or maps may be allowed

## Wide Column Stores

Query patterns
- Create, update or remove a row within a given column family
- Select rows according to a row key or simple conditions

Warning
- Wide column stores are not just a special kind of RDBMSs with a variable set of columns!


## Wide Column Stores

Suitable use cases
- Event logging, content management systems, blogs, …
    - I.e. for structured flat data with similar schema

When not to use
- ACID transactions are required
- Complex queries: aggregation (SUM, AVG, …), joining, …
- Early prototypes: i.e. when database design may change

Representatives
- Apache Cassandra, Apache HBase, Apache Accumulo, Hypertable, Google Bigtable

##  Wide Column Stores

<img src="https://pandaforme.gitbooks.io/introduction-to-cassandra/content/Screen%20Shot%202016-02-24%20at%2012.24.12.png">

## Wide Column Stores Use Cases

Column stores offer very high performance and a highly scalable architecture. Because they’re fast to load and query, they’ve been popular among companies and organizations dealing with big data, IoT, and user recommendation and personalization engines.

- Spotify uses Cassandra to store user profile attributes and metadata about artists, songs, etc. for better personalization
- Facebook initially built its revamped Messages on top of HBase, but is now also used for other Facebook services like the Nearby Friends feature and search indexing
- Outbrain uses Cassandra to serve over 190 billion personalized content recommendations each month

## Graph Databases

Data model
- Property graphs
    - Directed / undirected graphs, i.e. collections of …
        - nodes (vertices) for real-world entities, and
        - relationships (edges) between these nodes
    - Both the nodes and relationships can be associated with additional properties

Types of databases
- Non-transactional = small number of very large graphs
- Transactional = large number of small graphs

## Graph Databases

Query patterns
- Create, update or remove a node / relationship in a graph
- Graph algorithms (shortest paths, spanning trees, …)
- General graph traversals
- Sub-graph queries or super-graph queries
- Similarity based queries (approximate matching)

Representatives
- Neo4j, Titan, Apache Giraph, InfiniteGraph, FlockDB
- Multi-model: OrientDB, OpenLink Virtuoso, ArangoDB

## Graph Databases

Suitable use cases
- Social networks, routing, dispatch, and location-based, services, recommendation engines, chemical compounds, biological pathways, linguistic trees, …
    - I.e. simply for __graph structures__

When not to use
- Extensive batch operations are required
    - Multiple nodes / relationships are to be affected
- Only too large graphs to be stored
    - Graph distribution is difficult or impossible at all


## Graph Databases

The image below shows how a relational database like MySQL works, which use memory-intensive and more complicated join operations to search entire tables to find a match:

<img src="https://s3.amazonaws.com/dev.assets.neo4j.com/wp-content/uploads/from_relational_model.png">


## Graph Databases

Compare that to a graph database, which already predetermines relationships based on connected nodes, making queries much faster and more economical.

<img src="https://s3.amazonaws.com/dev.assets.neo4j.com/wp-content/uploads/relational_to_graph.png">


## Graph Databases Use Cases

Graph databases are great for establishing data relationships especially when dealing with large data sets. They offer blazing fast query performance that relational databases cannot compete with, especially as data grows much deeper.

- Walmart uses Neo4j to provide customers personalized, relevant product recommendations and promotions
- Medium uses Neo4j to build their social graph to enhance content personalization
- Cisco uses Neo4j to mine customer support cases to anticipate bugs

## Big Data

<img src="https://media.makeameme.org/created/big-data-big-5ad56d.jpg">

## What is Big Data

- high Volume
- high Velocity
- high Variety
- Veracity

## Data Sources

- Social media and networks
- Scientific instruments
- Mobile devices
- Sensor technology and networks

# Architecture Design Project

# Background

Our client is a well-known manufacturer who builds different types of robotic manipulators. These robots are used by different operators/companies all around the world. 

During the operation, each robot creates a log file which includes timestamps as well as information from different sensors.

For example, this is a snapshot of the log file

```
.
.
.
timestamp 10:20:00 X 100 Y 100 Z 100
timestamp 10:20:01 T 5 R 6
timestamp 10:20:02 X 101 Y 99 Z 99
timestamp 10:20:03 X 102 Y 100 Z 99
timestamp 10:20:04 T 7 R 6
timestamp 10:20:05 X 100 Y 100 Z 99
.
.
.

```

The logfile can contain differet sensors and sometimes, there are differences between robot sensors and how they are captured.

# Main Idea

Our client wants to create a platform (website) for health-condition-monitoring of the robots. In particular, our client wants to create a platform where different operators of the robotic arms can upload the log files and get insights about the status of their robot and different parts of it. 

<img src="photos/cybernetics-1869205_1280.jpg">

# Task

Design an architecture for the front-end and the back-end. (on the abstract level)
Consider that each logfile is around 100Mb and is in text format

- What types of databases are needed?
- In case of sql databases, what columns do you suggest?
- Do we need any nosql db? what type?

Note: in this case, there are no unique solution. Try to be creative :)